# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('../kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to abhinav14csu009@ncuindia.edu and will expire on August 16, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1482251963.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000002  | 3.223803     | 6847177.773672     | 380824.455891 |

| 10        | 11       | 0.000002  | 3.730116     | 6842123.232651     | 364204.576180 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [14]:
print model_all.coefficients['value'].nnz()
for i in model_all.coefficients:
    if i['value'] != 0.0:
        print i['name'] 

6
(intercept)
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [15]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [20]:
import numpy as np
least_err = []
for l1_penalty in np.logspace(1, 7, num=13):
    curr_model = graphlab.linear_regression.create(training, features=all_features, target='price', validation_set=None,
                                                   l1_penalty=l1_penalty, l2_penalty=0, verbose=False)
    err = curr_model.predict(validation) - validation['price']
    rss = (err**2).sum()
    if len(least_err) == 0:
        least_err.append(rss)
        least_err.append(l1_penalty)
    elif rss < least_err[0]:
        least_err[0] = rss
        least_err[1] = l1_penalty
print "Lowest RSS produced is " + str(least_err[0]) + " by penalty " + str(least_err[1])

Lowest RSS produced is 6.25766285142e+14 by penalty 10.0


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [21]:
low_rss_model = graphlab.linear_regression.create(training, features=all_features, target='price',
                                                 validation_set=None, l1_penalty=10.0, l2_penalty=0, verbose=False)
low_rss_model.coefficients['value'].nnz()

18

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [22]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [23]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [30]:
smallest_l1_penalty = []
largest_l1_penalty = []
found_smallest = False
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, features=all_features, target='price',
                                              l1_penalty=l1_penalty,l2_penalty=0, verbose=False,
                                              validation_set=None)
    non_zeroes = model['coefficients']['value'].nnz()
    
    print str(l1_penalty) + " gives " + str(non_zeroes) + " non-zeroes"
    
    if len(largest_l1_penalty) == 0 and len(smallest_l1_penalty) == 0:
        largest_l1_penalty.append(l1_penalty)
        largest_l1_penalty.append(non_zeroes)
        
        smallest_l1_penalty.append(l1_penalty)
        smallest_l1_penalty.append(non_zeroes)
    
    elif non_zeroes > max_nonzeros and l1_penalty > largest_l1_penalty[0]:
        largest_l1_penalty[0] = l1_penalty
        largest_l1_penalty[1] = non_zeroes
    
    if not found_smallest and non_zeroes < max_nonzeros:
        smallest_l1_penalty[0] = l1_penalty
        smallest_l1_penalty[1] = non_zeroes
        found_smallest = True

print "Largest L1 penalty is " + str(largest_l1_penalty[0]) + " with " + str(largest_l1_penalty[1]) + " non-zeroes"
print "Smallest L1 penalty is " + str(smallest_l1_penalty[0]) + " with " + str(smallest_l1_penalty[1]) + " non-zeroes"

100000000.0 gives 18 non-zeroes
127427498.57 gives 18 non-zeroes
162377673.919 gives 18 non-zeroes
206913808.111 gives 18 non-zeroes
263665089.873 gives 17 non-zeroes
335981828.628 gives 17 non-zeroes
428133239.872 gives 17 non-zeroes
545559478.117 gives 17 non-zeroes
695192796.178 gives 17 non-zeroes
885866790.41 gives 16 non-zeroes
1128837891.68 gives 15 non-zeroes
1438449888.29 gives 15 non-zeroes
1832980710.83 gives 13 non-zeroes
2335721469.09 gives 12 non-zeroes
2976351441.63 gives 10 non-zeroes
3792690190.73 gives 6 non-zeroes
4832930238.57 gives 5 non-zeroes
6158482110.66 gives 3 non-zeroes
7847599703.51 gives 1 non-zeroes
10000000000.0 gives 1 non-zeroes
Largest L1 penalty is 2976351441.63 with 10 non-zeroes
Smallest L1 penalty is 3792690190.73 with 6 non-zeroes


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [31]:
l1_penalty_min = smallest_l1_penalty[0]
l1_penalty_max = largest_l1_penalty[0]

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [32]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [33]:
rss_min = []
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, features=all_features, target='price',
                                              l1_penalty=l1_penalty, l2_penalty=0, validation_set=None,
                                              verbose=False)
    non_zeroes = model.coefficients['value'].nnz()
    
    if non_zeroes == max_nonzeros:
        err = model.predict(validation) - validation['price']
        rss = (err**2).sum()
        
        if len(rss_min) == 0:
            rss_min.append(rss)
            rss_min.append(l1_penalty)
            rss_min.append(non_zeroes)
        elif rss < rss_min[0]:
            rss_min[0] = rss
            rss_min[1] = l1_penalty
            rss_min[2] = non_zeroes

print "Lowest RSS is " + str(rss_min[0]) + " by " + str(rss_min[1]) + " penalty having " + str(rss_min[2]) + " non-zeroes"

Lowest RSS is 1.04693748875e+15 by 3448968612.16 penalty having 7 non-zeroes


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [34]:
tmp_model = graphlab.linear_regression.create(training, features=all_features, target='price',
                                              l1_penalty=3448968612.16, l2_penalty=0, validation_set=None,
                                              verbose=False)
for i in tmp_model.coefficients:
    if i['value'] != 0.0:
        print i['name']

(intercept)
bedrooms
bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above
